# Convergence speed analysis

This notebook assumes the user is in his own machine.

**Change this if necessary:**

In [ ]:
USER = "st76i1_1"
GROUP = "st76i1"

### Generate runs

In [8]:
# set .env
# make upload
# run optimizer.main in ssh
# make get_logs

/home/lucca/miniconda3/bin/python3: Relative module names not supported
